# 한영 번역 서비스 만들기 M2M100

In [2]:
# !pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.6 MB/s eta 0:00:00


In [2]:
# !pip install g2pk

In [4]:
# !pip install datasets

In [7]:
# !pip install soundfile

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 37.9 MB/s eta 0:00:00


In [3]:
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer

ko_text = "이것은 한/영 번역기 입니다. m2m100모델로 만들었어요"
en_text = "This is a Korean to English translator. I made it using m2m100_418m."

model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M")
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# translate Hindi to French
tokenizer.src_lang = "ko"
encoded_ko = tokenizer(ko_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_ko, forced_bos_token_id=tokenizer.get_lang_id("en"))
translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
# => "La vie est comme une boîte de chocolat."
print(translated_text)

# translate Chinese to English
tokenizer.src_lang = "en"
encoded_en = tokenizer(en_text, return_tensors="pt")
generated_tokens = model.generate(**encoded_en, forced_bos_token_id=tokenizer.get_lang_id("ko"))
translated_text2 = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
# => "Life is like a box of chocolate."

print(translated_text2)

This is one/sing translator. made with m2m100 model
이것은 한국어에서 영어로 번역가입니다. m2m100_418m를 사용하여 만들었습니다.


### 다국어 번역기

In [3]:
import gradio as gr
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer
import torch_directml  # Import torch-directml for DirectML device
import torch  # Import PyTorch

#Initialize the DirectML device
dml_device = torch_directml.device()

#Initialize the model and tokenizer
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to(dml_device)
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

#List of available languages with their language codes
languages = {
    "Korean": "ko",
    "English": "en",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Japanese": "ja",
    "Chinese (Simplified)": "zh",
    "Russian": "ru",
    # Add more languages as needed
}

def translate(text, source_lang, target_lang):
    tokenizer.src_lang = languages[source_lang]
    encoded_text = tokenizer(text, return_tensors="pt").to(dml_device)  # Send input tensors to DirectML device
    generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.get_lang_id(languages[target_lang]))
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    return translated_text

with gr.Blocks() as app:
    gr.Markdown("# Multilingual Translation Service")

    with gr.Row():
        source_lang = gr.Dropdown(label="Source Language", choices=list(languages.keys()), value="Korean")
        target_lang = gr.Dropdown(label="Target Language", choices=list(languages.keys()), value="English")

    input_text = gr.Textbox(label="Input Text")
    output_text = gr.Textbox(label="Translated Text")
    translate_btn = gr.Button("Translate")

    translate_btn.click(fn=translate, inputs=[input_text, source_lang, target_lang], outputs=output_text)

app.launch(inline=False, share=True)

/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Parallel'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/gradio_client/documentation.py:106: UserWarning: Could not get documentation group for <class 'gradio.mix.Series'>: No known documentation group for module 'gradio.mix'
  warnings.warn(f"Could not get documentation group for {cls}: {exc}")
/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `cle

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://b443b838471f5fb246.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


### 번역된 곳 발음도 같이 나오게

In [8]:
import gradio as gr
from transformers import M2M100ForConditionalGeneration, M2M100Tokenizer, SpeechT5Processor, SpeechT5ForTextToSpeech, SpeechT5HifiGan
from datasets import load_dataset
import torch_directml  # Import torch-directml for DirectML device
import torch  # Import PyTorch
import soundfile as sf

# Initialize the DirectML device
dml_device = torch_directml.device()

# Initialize the translation model and tokenizer
model = M2M100ForConditionalGeneration.from_pretrained("facebook/m2m100_418M").to(dml_device)
tokenizer = M2M100Tokenizer.from_pretrained("facebook/m2m100_418M")

# List of available languages with their language codes
languages = {
    "Korean": "ko",
    "English": "en",
    "French": "fr",
    "Spanish": "es",
    "German": "de",
    "Japanese": "ja",
    "Chinese (Simplified)": "zh",
    "Russian": "ru",
}

# Load the Microsoft SpeechT5 TTS model
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
speech_model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")
vocoder = SpeechT5HifiGan.from_pretrained("microsoft/speecht5_hifigan")

# Load speaker embeddings dataset for voice characteristics
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")

# Translation function with TTS integration
def translate_and_tts(text, source_lang, target_lang):
    tokenizer.src_lang = languages[source_lang]
    encoded_text = tokenizer(text, return_tensors="pt").to(dml_device)  # Send input tensors to DirectML device
    generated_tokens = model.generate(**encoded_text, forced_bos_token_id=tokenizer.get_lang_id(languages[target_lang]))
    translated_text = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]
    
    # If target language is English, perform TTS
    tts_output_file = None
    if target_lang == "English":
        tts_output_file = convert_text_to_speech(translated_text)
    
    return translated_text, tts_output_file

# Text-to-speech conversion using Microsoft SpeechT5 TTS
def convert_text_to_speech(text):
    inputs = processor(text=text, return_tensors="pt")
    
    # Choose a speaker embedding by index or select randomly
    speaker_embeddings = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)
    
    # Generate the speech using the TTS model and vocoder
    speech = speech_model.generate_speech(inputs["input_ids"], speaker_embeddings, vocoder=vocoder)
    
    # Save the generated speech to a file
    temp_audio_file = "output_speech.wav"
    sf.write(temp_audio_file, speech.numpy(), samplerate=16000)
    
    return temp_audio_file

with gr.Blocks() as app:
    gr.Markdown("# Multilingual Translation Service with Microsoft SpeechT5 TTS")

    with gr.Row():
        source_lang = gr.Dropdown(label="Source Language", choices=list(languages.keys()), value="Korean")
        target_lang = gr.Dropdown(label="Target Language", choices=list(languages.keys()), value="English")

    input_text = gr.Textbox(label="Input Text")
    output_text = gr.Textbox(label="Translated Text")
    tts_audio = gr.Audio(label="Text-to-Speech Output")  # Audio player for speech output
    translate_btn = gr.Button("Translate and Speak")

    # Connect the button click event to the translation and TTS function
    translate_btn.click(fn=translate_and_tts, inputs=[input_text, source_lang, target_lang], outputs=[output_text, tts_audio])

app.launch(inline=False, share=True)


/home/user/miniforge3/envs/torch/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Generating validation split: 100%|███████████████████████████| 7931/7931 [00:00<00:00, 125382.20 examples/s]


Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.40.0, however version 4.29.0 is available, please upgrade.
--------
Running on public URL: https://28931016c641ec60e5.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [4]:
app.close()

Closing server running on port: 7861
